# Resume vs Job Description Evaluator
## Production AI Recruiter Agent with LangChain + LangGraph + Local Qwen

**Features:**
- 🤖 Uses Qwen2.5-3B-Instruct (runs locally on Colab GPU)
- 🔍 RAG architecture with FAISS vector database
- 📊 Multi-step agent workflow with LangGraph
- 💰 No API costs - completely free
- 🎯 Production-ready evaluation system

**Fixed:** Resolved numpy compatibility issues

---

## CELL 1: Install Dependencies

Install all required packages with compatible versions

In [2]:
# # First: Uninstall problematic packages
# !pip uninstall -y jax jaxlib numpy -q

# # Install numpy FIRST with correct version
# !pip install -q numpy==1.24.3

# # Install PyTorch
# !pip install -q torch==2.1.0 --index-url https://download.pytorch.org/whl/cu118

# # Install transformers ecosystem (compatible versions)
# !pip install -q transformers==4.35.2
# !pip install -q accelerate==0.24.1
# !pip install -q bitsandbytes==0.41.3.post2
# !pip install -q sentencepiece==0.1.99

# # Install sentence-transformers
# !pip install -q sentence-transformers==2.2.2

# # Install FAISS
# !pip install -q faiss-cpu==1.7.4

# # Install LangChain (use older compatible versions)
# !pip install -q langchain==0.0.350
# !pip install -q langchain-community==0.0.1
# !pip install -q langgraph==0.0.19

# # Verify installations
# import numpy
# import torch
# print(f"✅ All dependencies installed successfully!")
# print(f"NumPy version: {numpy.__version__}")
# print(f"PyTorch version: {torch.__version__}")

In [5]:
!pip install -q --upgrade pip

!pip install -q \
torch==2.3.1 torchvision torchaudio \
transformers==4.41.2 \
accelerate==0.30.1 \
bitsandbytes==0.43.1 \
sentence-transformers \
faiss-cpu \
langchain \
langchain-core \
langchain-community \
langchain-text-splitters \
langgraph

In [6]:
import torch, transformers

print(torch.__version__)
print(transformers.__version__)
print(torch.cuda.is_available())

2.3.1+cu121
4.41.2
True


## CELL 2: Import Required Libraries

Import all necessary modules - now compatible!

In [7]:
!pip install -U \
accelerate==0.33.0 \
peft==0.11.1 \
transformers==4.41.2 \
sentence-transformers==2.2.2 \
huggingface_hub==0.23.0

  Using cached accelerate-0.33.0-py3-none-any.whl.metadata (18 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
Using cached accelerate-0.33.0-py3-none-any.whl (315 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.2.2
    Uninstalling sentence-transformers-5.2.2:
      Successfully uninstalled sentence-transformers-5.2.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [sentence-transformers]


In [8]:
# Core Python libraries
import os
import re
import json
import warnings
from typing import TypedDict, List, Dict, Any, Optional

# NumPy and PyTorch
import numpy as np
import torch

# Transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)

# LangChain (NEW)
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document

from langchain_text_splitters import RecursiveCharacterTextSplitter

# LangGraph
from langgraph.graph import StateGraph, END

# Suppress warnings
warnings.filterwarnings('ignore')
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

print("✅ All imports successful!")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"📊 NumPy version: {np.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")

✅ All imports successful!
🔥 PyTorch version: 2.3.1+cu121
📊 NumPy version: 1.26.4
🎮 CUDA available: True
🎮 GPU: Tesla T4


## CELL 3: Load Qwen LLM Locally

Load Qwen2.5-3B-Instruct model with 4-bit quantization.

In [9]:
# Model configuration
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"

print(f"📥 Loading {MODEL_NAME}...")
print("⏳ This may take 2-3 minutes on first run...")

# Configure 4-bit quantization
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_use_double_quant=True,
# )

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Create pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,  # Low temperature for consistency
    top_p=0.95,
    repetition_penalty=1.15,
)

# Wrap with LangChain
llm = HuggingFacePipeline(pipeline=pipe)

print("\n✅ Qwen model loaded successfully!")
print(f"📊 Model size: ~3B parameters (4-bit quantized)")
print(f"💾 GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

📥 Loading Qwen/Qwen2.5-3B-Instruct...
⏳ This may take 2-3 minutes on first run...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


✅ Qwen model loaded successfully!
📊 Model size: ~3B parameters (4-bit quantized)
💾 GPU memory allocated: 6.89 GB


## CELL 4: Define AgentState

State structure for the workflow.

In [10]:
class AgentState(TypedDict):
    """State object for the agent workflow."""
    jd_text: str
    resume_text: str
    jd_docs: Optional[List[Document]]
    resume_docs: Optional[List[Document]]
    jd_vs: Optional[FAISS]
    resume_vs: Optional[FAISS]
    requirements: Optional[List[Dict[str, Any]]]
    evaluations: Optional[List[Dict[str, Any]]]
    final_report: Optional[Dict[str, Any]]

print("✅ AgentState defined!")

✅ AgentState defined!


## CELL 5: Text Chunking

Split documents into chunks.

In [11]:
def chunk_text(text: str, chunk_size: int = 500, chunk_overlap: int = 50) -> List[Document]:
    """Split text into chunks."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", ". ", " ", ""],
    )

    chunks = text_splitter.split_text(text)
    documents = [
        Document(page_content=chunk, metadata={"chunk_id": i})
        for i, chunk in enumerate(chunks)
    ]

    return documents

print("✅ Chunking function ready!")

✅ Chunking function ready!


## CELL 6: FAISS Vector Store

Build vector indexes.

In [12]:
!pip install -U sentence-transformers

  Using cached sentence_transformers-5.2.2-py3-none-any.whl.metadata (16 kB)
Using cached sentence_transformers-5.2.2-py3-none-any.whl (494 kB)
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [14]:
print("📥 Loading embedding model...")

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

def build_vector_store(documents: List[Document]) -> FAISS:
    """Build FAISS index."""
    if not documents:
        raise ValueError("No documents provided")
    return FAISS.from_documents(documents=documents, embedding=embeddings)

print("✅ Vector store ready!")

📥 Loading embedding model...


.gitattributes: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

✅ Vector store ready!


## CELL 7: Requirement Extraction Node

In [15]:
def extract_requirements_node(state: AgentState) -> AgentState:
    """Extract requirements from JD."""
    print("\n🔍 Extracting requirements...")

    prompt_template = """Extract ALL requirements from this job description.

Job Description:
{jd_text}

Return a JSON array:
[
  {{"requirement": "5+ years Python", "category": "experience", "importance": "critical"}},
  {{"requirement": "Bachelor's CS", "category": "education", "importance": "critical"}}
]

Categories: skills, education, experience, tools
Importance: critical, preferred

Return ONLY the JSON array."""

    prompt = PromptTemplate(template=prompt_template, input_variables=["jd_text"])
    chain = prompt | llm
    response = chain.invoke({"jd_text": state['jd_text'][:2000]})

    try:
        json_text = response.strip()
        if "```json" in json_text:
            json_text = json_text.split("```json")[1].split("```")[0].strip()
        json_match = re.search(r'\[.*\]', json_text, re.DOTALL)
        if json_match:
            json_text = json_match.group(0)
        requirements = json.loads(json_text)
        if not isinstance(requirements, list):
            requirements = [requirements]
        print(f"✅ Extracted {len(requirements)} requirements")
    except:
        requirements = [{"requirement": "General qualifications", "category": "experience", "importance": "critical"}]
        print("⚠️  Using fallback requirements")

    state['requirements'] = requirements
    return state

print("✅ Extraction node defined!")

✅ Extraction node defined!


## CELL 8: Resume Evaluation Node

In [16]:
def evaluate_resume_node(state: AgentState) -> AgentState:
    """Evaluate each requirement with RAG."""
    print("\n⚖️  Evaluating resume...")

    requirements = state['requirements']
    resume_vs = state['resume_vs']
    evaluations = []

    for i, req in enumerate(requirements[:10], 1):  # Limit to 10 for speed
        requirement_text = req['requirement']
        print(f"  [{i}/{min(10, len(requirements))}] {requirement_text[:40]}...")

        try:
            relevant_chunks = resume_vs.similarity_search(requirement_text, k=3)
            evidence = "\n\n".join([doc.page_content for doc in relevant_chunks])
        except:
            evidence = "No evidence found."

        eval_prompt = """Evaluate if candidate meets this requirement.

Requirement: {requirement}
Evidence: {evidence}

Return JSON:
{{"match": "yes", "confidence": "high", "score": 5, "evidence_summary": "...", "reasoning": "..."}}

match: yes/partial/no
confidence: high/medium/low
score: 0-5

Return ONLY JSON."""

        prompt = PromptTemplate(template=eval_prompt, input_variables=["requirement", "evidence"])
        chain = prompt | llm
        response = chain.invoke({"requirement": requirement_text, "evidence": evidence[:1000]})

        try:
            json_text = response.strip()
            if "```json" in json_text:
                json_text = json_text.split("```json")[1].split("```")[0].strip()
            json_match = re.search(r'\{.*\}', json_text, re.DOTALL)
            if json_match:
                json_text = json_match.group(0)
            evaluation = json.loads(json_text)
            evaluation['requirement'] = requirement_text
            evaluation['category'] = req['category']
            evaluation['importance'] = req['importance']
            print(f"    ✓ {evaluation.get('match', 'N/A')} | {evaluation.get('score', 0)}/5")
        except:
            evaluation = {
                'requirement': requirement_text,
                'category': req['category'],
                'importance': req['importance'],
                'match': 'no',
                'score': 0,
                'confidence': 'low',
                'evidence_summary': 'Parse error',
                'reasoning': 'Failed'
            }

        evaluations.append(evaluation)

    print(f"✅ Evaluated {len(evaluations)} requirements")
    state['evaluations'] = evaluations
    return state

print("✅ Evaluation node defined!")

✅ Evaluation node defined!


## CELL 9: Final Decision Node

In [17]:
def final_decision_node(state: AgentState) -> AgentState:
    """Make final decision."""
    print("\n📊 Final decision...")

    evaluations = state['evaluations']
    total = len(evaluations)
    avg_score = sum(e.get('score', 0) for e in evaluations) / max(total, 1)
    fit_score = int((avg_score / 5.0) * 100)

    critical = [e for e in evaluations if e.get('importance') == 'critical']
    met_critical = sum(1 for e in critical if e.get('match') == 'yes')

    decision_prompt = """Based on evaluations, make hiring decision.

Stats: {total} requirements, {met_critical}/{total_critical} critical met, avg score {avg_score:.1f}/5

Return JSON:
{{"fit_score": 85, "decision": "ACCEPT", "strengths": [...], "gaps": [...], "summary": "..."}}

decision: ACCEPT (score>=70, all critical), MAYBE (50-69), REJECT (<50)

Return ONLY JSON."""

    prompt = PromptTemplate(template=decision_prompt, input_variables=["total", "met_critical", "total_critical", "avg_score"])
    chain = prompt | llm
    response = chain.invoke({"total": total, "met_critical": met_critical, "total_critical": len(critical), "avg_score": avg_score})

    try:
        json_text = response.strip()
        if "```json" in json_text:
            json_text = json_text.split("```json")[1].split("```")[0].strip()
        json_match = re.search(r'\{.*\}', json_text, re.DOTALL)
        if json_match:
            json_text = json_match.group(0)
        final_report = json.loads(json_text)
        print(f"✅ {final_report.get('decision', 'N/A')} | {final_report.get('fit_score', 0)}/100")
    except:
        final_report = {
            'fit_score': fit_score,
            'decision': 'ACCEPT' if fit_score >= 70 else 'MAYBE' if fit_score >= 50 else 'REJECT',
            'strengths': ['See evaluations'],
            'gaps': ['See evaluations'],
            'summary': f'Score: {fit_score}/100'
        }

    final_report['total_requirements'] = total
    final_report['critical_met'] = f"{met_critical}/{len(critical)}"
    state['final_report'] = final_report
    return state

print("✅ Decision node defined!")

✅ Decision node defined!


## CELL 10: Build LangGraph Workflow

In [18]:
def ingest_node(state: AgentState) -> AgentState:
    print("\n📄 Ingesting...")
    state['jd_docs'] = chunk_text(state['jd_text'])
    state['resume_docs'] = chunk_text(state['resume_text'])
    print(f"  ✓ JD: {len(state['jd_docs'])} chunks")
    print(f"  ✓ Resume: {len(state['resume_docs'])} chunks")
    return state

def index_node(state: AgentState) -> AgentState:
    print("\n🔍 Indexing...")
    state['jd_vs'] = build_vector_store(state['jd_docs'])
    state['resume_vs'] = build_vector_store(state['resume_docs'])
    print("  ✓ Indexes built")
    return state

# Build graph
workflow = StateGraph(AgentState)
workflow.add_node("ingest", ingest_node)
workflow.add_node("index", index_node)
workflow.add_node("extract_requirements", extract_requirements_node)
workflow.add_node("evaluate_resume", evaluate_resume_node)
workflow.add_node("final_decision", final_decision_node)

workflow.set_entry_point("ingest")
workflow.add_edge("ingest", "index")
workflow.add_edge("index", "extract_requirements")
workflow.add_edge("extract_requirements", "evaluate_resume")
workflow.add_edge("evaluate_resume", "final_decision")
workflow.add_edge("final_decision", END)

app = workflow.compile()

print("✅ Workflow ready!")

✅ Workflow ready!


## CELL 11: Run Example

In [20]:
# Sample data
sample_jd = """Senior ML Engineer

Required:
- Master's in CS/AI
- 5+ years ML experience
- Python, TensorFlow, PyTorch
- Production ML deployment
- Cloud (AWS/GCP/Azure)
"""

sample_resume = """JOHN DOE - Senior ML Engineer

EDUCATION
Master's in Computer Science - Stanford 2018
Bachelor's Computer Engineering - UC Berkeley 2016

EXPERIENCE
Senior ML Engineer | Tech Corp | 2021-Present
- Built recommendation system (10M users)
- Deployed models on cloud  AWS SageMaker
- Python, TensorFlow, PyTorch
- Mentored 4 engineers

ML Engineer | DataTech | 2019-2021
- NLP models (92% accuracy)
- Computer vision systems
- Optimized ML pipelines
"""

def run_evaluation(jd_text: str, resume_text: str):
    print("="*80)
    print("🚀 STARTING EVALUATION")
    print("="*80)

    initial_state = {
        'jd_text': jd_text,
        'resume_text': resume_text,
        'jd_docs': None,
        'resume_docs': None,
        'jd_vs': None,
        'resume_vs': None,
        'requirements': None,
        'evaluations': None,
        'final_report': None
    }

    final_state = app.invoke(initial_state)

    print("\n" + "="*80)
    print("✅ COMPLETE")
    print("="*80)

    return final_state['final_report']

report = run_evaluation(sample_jd, sample_resume)

🚀 STARTING EVALUATION

📄 Ingesting...
  ✓ JD: 1 chunks
  ✓ Resume: 1 chunks

🔍 Indexing...
  ✓ Indexes built

🔍 Extracting requirements...
⚠️  Using fallback requirements

⚖️  Evaluating resume...
  [1/1] General qualifications...
    ✓ yes | 3/5
✅ Evaluated 1 requirements

📊 Final decision...
✅ ACCEPT | 85/100

✅ COMPLETE


## CELL 12: Display Report

In [21]:
def display_report(report: Dict[str, Any]):
    print("\n" + "="*80)
    print("📊 FINAL REPORT")
    print("="*80)

    decision_emoji = {'ACCEPT': '✅', 'MAYBE': '⚠️', 'REJECT': '❌'}.get(report.get('decision', 'N/A'), '❓')

    print(f"\n{decision_emoji} DECISION: {report.get('decision', 'N/A')}")
    print(f"📈 FIT SCORE: {report.get('fit_score', 0)}/100")
    print(f"\n📋 Total: {report.get('total_requirements', 0)}")
    print(f"🎯 Critical Met: {report.get('critical_met', 'N/A')}")

    if report.get('strengths'):
        print("\n💪 STRENGTHS:")
        for s in report['strengths']:
            print(f"  • {s}")

    if report.get('gaps'):
        print("\n⚠️  GAPS:")
        for g in report['gaps']:
            print(f"  • {g}")

    print(f"\n📝 SUMMARY:\n  {report.get('summary', 'N/A')}")
    print("\n" + "="*80)
    print("\nFull JSON:")
    print(json.dumps(report, indent=2))

display_report(report)


📊 FINAL REPORT

✅ DECISION: ACCEPT
📈 FIT SCORE: 85/100

📋 Total: 1
🎯 Critical Met: 1/1

💪 STRENGTHS:
  • The candidate has met the sole critical requirement.

⚠️  GAPS:
  • {'requirement_id': 1, 'description': 'Other areas require improvement.'}

📝 SUMMARY:
  Despite meeting the sole critical requirement, overall performance needs enhancement across multiple areas.


Full JSON:
{
  "fit_score": 85,
  "decision": "ACCEPT",
  "strengths": [
    "The candidate has met the sole critical requirement."
  ],
  "gaps": [
    {
      "requirement_id": 1,
      "description": "Other areas require improvement."
    }
  ],
  "summary": "Despite meeting the sole critical requirement, overall performance needs enhancement across multiple areas.",
  "total_requirements": 1,
  "critical_met": "1/1"
}


## Evaluation metrics

In [22]:
!pip install -q scikit-learn numpy

In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Hard-coded example
y_true = [1]   # Ground truth: ACCEPT
y_pred = [1]   # Model predicted: ACCEPT

# Metrics
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Force 2x2 confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=[0,1])
tn, fp, fn, tp = cm.ravel()

classification_metrics = {
    "accuracy": round(acc, 3),
    "precision": round(prec, 3),
    "recall": round(rec, 3),
    "f1_score": round(f1, 3)
}

business_metrics = {
    "false_positive_rate": round(fp / (fp + tn + 1e-9), 3),
    "false_negative_rate": round(fn / (fn + tp + 1e-9), 3),
    "time_saved_hours": 0.25,
    "cost_saved_usd": 0.02
}

print("\n" + "="*60)
print("📊 EVALUATION METRICS (HARDCODED SAMPLE)")
print("="*60)

print("\n🎯 Classification Metrics:")
for k, v in classification_metrics.items():
    print(f"{k}: {v}")

print("\n💼 Business Metrics:")
for k, v in business_metrics.items():
    print(f"{k}: {v}")


📊 EVALUATION METRICS (HARDCODED SAMPLE)

🎯 Classification Metrics:
accuracy: 1.0
precision: 1.0
recall: 1.0
f1_score: 1.0

💼 Business Metrics:
false_positive_rate: 0.0
false_negative_rate: 0.0
time_saved_hours: 0.25
cost_saved_usd: 0.02


In [27]:
y_true = [1, 0, 1, 1, 0, 0, 1]
y_pred = [1, 0, 1, 0, 0, 1, 1]

In [28]:
metrics, business = evaluate_classification(y_true, y_pred)

print("📊 Classification Metrics:")
print(metrics)

print("\n💼 Business Metrics:")
print(business)

📊 Classification Metrics:
{'accuracy': 0.714, 'precision': 0.75, 'recall': 0.75, 'f1_score': 0.75}

💼 Business Metrics:
{'false_positive_rate': 0.333, 'false_negative_rate': 0.25, 'time_saved_hours': 1.8, 'cost_saved_usd': 0.14}


## 👤 Author

**Name:** Akbarjon Maytrasulov

🔗 GitHub: https://github.com/Matrasulov  
🔗 LinkedIn: https://www.linkedin.com/in/akbarjon-matrasulov-462667210/

---

## 📌 Project Summary
This notebook demonstrates an Agentic AI Resume Evaluator using:

- Qwen LLM (local inference)
- LangChain + LangGraph
- RAG-based resume & job description matching
- FAISS vector database
- Embedding-based semantic comparison

The system analyzes how well a candidate's resume matches a job description and produces a structured hiring recommendation.

---

## 💡 Notes
This project was built as part of an NLP/Agentic AI portfolio to showcase:
- Retrieval-Augmented Generation (RAG)
- Local LLM deployment
- Resume–Job semantic matching
- AI-based evaluation pipelines